In [3]:
import sys 
sys.path.insert(0, '../')

In [4]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from common.common import create_folder
from common.pytorch import load_model
import pytorch_pretrained_bert as Bert
from model.utils import age_vocab
from common.common import load_obj
from dataLoader.MLM_med import MLMLoader
from torch.utils.data import DataLoader
import pandas as pd
from model.MLM_med import BertForMaskedLM
from model.optimiser import adam
import sklearn.metrics as skm
import numpy as np
import torch
import time
import torch.nn as nn




In [3]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.med_vocab_size = config.get('med_vocab_size')  # OWN EMBEDDINGS
        
class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

In [5]:
file_config = {
    'vocab':'token2idx-added',  # vocabulary idx2token, token2idx
    'med_vocab' : 'med-token2idx', 
    'data': './behrt_format_mimic4ed/',  # formated data 
    'model_path': 'josa_model', # where to save model
    'model_name': 'test_model', # model name
    'file_name': 'test_model',  # log path
}
create_folder(file_config['model_path'])

In [8]:
global_params = {
    'max_seq_len': 64,
    'max_age': 110,
    'month': 1, # try to transform data into month-based
    'age_symbol': None,
    'min_visit': 5, # try to reduce to 3
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 256,
    'use_cuda': False,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cpu'
}

In [11]:
BertVocab = load_obj(file_config['vocab'])
med_BertVocab = load_obj(file_config['med_vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon=global_params['month'], symbol=global_params['age_symbol'])
print(ageVocab)

{'PAD': 0, 'UNK': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11, '10': 12, '11': 13, '12': 14, '13': 15, '14': 16, '15': 17, '16': 18, '17': 19, '18': 20, '19': 21, '20': 22, '21': 23, '22': 24, '23': 25, '24': 26, '25': 27, '26': 28, '27': 29, '28': 30, '29': 31, '30': 32, '31': 33, '32': 34, '33': 35, '34': 36, '35': 37, '36': 38, '37': 39, '38': 40, '39': 41, '40': 42, '41': 43, '42': 44, '43': 45, '44': 46, '45': 47, '46': 48, '47': 49, '48': 50, '49': 51, '50': 52, '51': 53, '52': 54, '53': 55, '54': 56, '55': 57, '56': 58, '57': 59, '58': 60, '59': 61, '60': 62, '61': 63, '62': 64, '63': 65, '64': 66, '65': 67, '66': 68, '67': 69, '68': 70, '69': 71, '70': 72, '71': 73, '72': 74, '73': 75, '74': 76, '75': 77, '76': 78, '77': 79, '78': 80, '79': 81, '80': 82, '81': 83, '82': 84, '83': 85, '84': 86, '85': 87, '86': 88, '87': 89, '88': 90, '89': 91, '90': 92, '91': 93, '92': 94, '93': 95, '94': 96, '95': 97, '96': 98, '97': 99, '98': 100, '99': 

In [15]:
data = pd.read_parquet(file_config['data'])
# remove patients with visits less than min visit
# print(data)
data['length'] = data['icd_code'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
data = data.reset_index(drop=True)

      subject_id                                           icd_code  \
0       10000032  [5728, 78959, 07070, V08, SEP, 78959, 07070, 5...   
1       10000980  [4111, 78609, 41400, 25001, 4019, SEP, 4280, 4...   
2       10001884  [78650, 4019, 49390, SEP, 78650, 4019, SEP, 49...   
3       10002013  [78650, 5939, SEP, 41091, SEP, 78650, 25000, V...   
4       10002428  [78791, 5589, 4019, 2720, SEP, 51881, 5119, 48...   
...          ...                                                ...   
6925    19994592  [29570, 2967, SEP, 311, V6284, SEP, R000, SEP,...   
6926    19997367  [6930, E9308, V103, SEP, 78609, 2851, 5789, 42...   
6927    19997911  [78701, 78791, 42789, 4589, SEP, 78791, SEP, K...   
6928    19998330  [51882, 78605, 496, 42731, SEP, 51881, 49121, ...   
6929    19998444  [5770, SEP, 78909, 78703, SEP, 78906, SEP, 789...   

                                      age_on_admittance  length  
0     [52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 5...       5  
1     [73, 73, 

In [8]:
# print(data)
Dset = MLMLoader(data, BertVocab['token2idx'], age2idx = ageVocab, med_token2idx=med_BertVocab['token2idx'], max_len=train_params['max_len_seq'], code='icd_code', age='age_on_admittance', med='med')

trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

In [9]:
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()), # number of disease + symbols for word embedding
    'med_vocab_size': len(med_BertVocab['token2idx'].keys()), # OWN EMBEDDINGS

    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()), # number of vocab for age embedding
    'max_position_embedding': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 6, # number of multi-head attention layers required
    'num_attention_heads': 12, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

print(model_config['vocab_size'])
print(model_config['max_position_embedding'])

79917
64


In [10]:
conf = BertConfig(model_config)
model = BertForMaskedLM(conf)


In [11]:
model = model.to(train_params['device'])
optim = adam(params=list(model.named_parameters()), config=optim_param)

t_total value of -1 results in schedule not being applied


In [12]:
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision

In [13]:
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt= 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt +=1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, input_ids, posi_ids, segment_ids, attMask, masked_label = batch
        print(age_ids)
        print(input_ids)
        loss, pred, label = model(input_ids, age_ids, segment_ids, posi_ids,attention_mask=attMask, masked_lm_labels=masked_label)
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 200==0:
            print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss/2000, cal_acc(label, pred), time.time()-start))
            temp_loss = 0
            start = time.time()
            
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()
            

    print("** ** * Saving fine - tuned model ** ** * ")
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    create_folder(file_config['model_path'])
    output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    torch.save(model_to_save.state_dict(), output_model_file)
        
    cost = time.time() - start
    return tr_loss, cost

In [14]:
f = open(os.path.join(file_config['model_path'], file_config['file_name']), "w")
f.write('{}\t{}\t{}\n'.format('epoch', 'loss', 'time'))
for e in range(50):
    loss, time_cost = train(e, trainload)
    loss = loss/data_len
    f.write('{}\t{}\t{}\n'.format(e, loss, time_cost))
f.close()    

tensor([[80, 80, 80,  ...,  0,  0,  0],
        [77, 77, 77,  ...,  0,  0,  0],
        [74, 74, 74,  ...,  0,  0,  0],
        ...,
        [49, 49, 49,  ...,  0,  0,  0],
        [90, 90, 90,  ...,  0,  0,  0],
        [51, 51, 51,  ...,  0,  0,  0]])
tensor([[    1,     3,     3,  ...,     4,     4,     4],
        [    1,     3,     3,  ...,     4,     4,     4],
        [    2,     3,     3,  ...,     4,     4,     4],
        ...,
        [    1,     3,  7036,  ...,     4,     4,     4],
        [    1, 15601,     0,  ...,     4,     4,     4],
        [    1,     3,     0,  ...,     4,     4,     4]])


IndexError: index out of range in self